<a href="https://colab.research.google.com/github/sussie78/Sentiment-Analysis-LLM/blob/main/Sentiment_Analysis_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/sentiment_dataset_final.csv")

# Check data
print(df.head())


                                         Text Sentiment  \
0     Too expensive for the quality provided.  Negative   
1          The packaging was nice and secure.  Negative   
2  Poor build quality, already falling apart.  Positive   
3   Difficult to set up and confusing to use.  Negative   
4           Awful smell, would not buy again.  Negative   

                          Processed_Text  
0              too expens qualiti provid  
1                      packag nice secur  
2  poor build qualiti alreadi fall apart  
3               difficult set confus use  
4               aw smell would buy again  


In [4]:
from sklearn.utils import resample

# Separate classes
df_negative = df[df["Sentiment"] == "Negative"]
df_positive = df[df["Sentiment"] == "Positive"]

# Oversample Positive class
df_positive_oversampled = resample(df_positive,
                                   replace=True,
                                   n_samples=len(df_negative),
                                   random_state=42)

# Combine and shuffle
df_balanced = pd.concat([df_negative, df_positive_oversampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check class distribution
print(df_balanced['Sentiment'].value_counts(normalize=True))


Sentiment
Positive    0.5
Negative    0.5
Name: proportion, dtype: float64


In [5]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced["Text"], df_balanced["Sentiment"], test_size=0.2, random_state=42
)


In [6]:
from transformers import DistilBertTokenizer

# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Encode labels
label_map = {"Negative": 0, "Positive": 1}
y_train_encoded = [label_map[label] for label in y_train]
y_test_encoded = [label_map[label] for label in y_test]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=128)
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Create datasets
train_dataset = SentimentDataset(X_train, y_train_encoded)
test_dataset = SentimentDataset(X_test, y_test_encoded)


In [8]:
from transformers import DistilBertForSequenceClassification

# Load model (2 output labels: Positive & Negative)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Start training
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bloodofjesusblessed (bloodofjesusblessed-not-applicable) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.682600,0.742847
2,0.617900,0.880003
3,0.585400,0.865771


TrainOutput(global_step=69, training_loss=0.6034359206324038, metrics={'train_runtime': 706.7623, 'train_samples_per_second': 0.386, 'train_steps_per_second': 0.098, 'total_flos': 918216402012.0, 'train_loss': 0.6034359206324038, 'epoch': 3.0})

In [11]:
# Get predictions
preds = trainer.predict(test_dataset)

# Convert logits to class labels
pred_labels = torch.argmax(torch.tensor(preds.predictions), dim=1).tolist()

# Map back to sentiment labels
pred_labels = ["Negative" if lbl == 0 else "Positive" for lbl in pred_labels]

# Print accuracy
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, pred_labels))
print(classification_report(y_test, pred_labels))


Accuracy: 0.391304347826087
              precision    recall  f1-score   support

    Negative       0.43      0.50      0.46        12
    Positive       0.33      0.27      0.30        11

    accuracy                           0.39        23
   macro avg       0.38      0.39      0.38        23
weighted avg       0.38      0.39      0.38        23



In [12]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


('sentiment_model/tokenizer_config.json',
 'sentiment_model/special_tokens_map.json',
 'sentiment_model/vocab.txt',
 'sentiment_model/added_tokens.json')

In [13]:
from transformers import pipeline, DistilBertForSequenceClassification, DistilBertTokenizer

# Define model path
model_path = "/content/sentiment_model"

# Load trained model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Create a classification pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Test the model
print(classifier("I love this product!"))  # Expected Output: Positive or Negative


Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.5137859582901001}]
